# Deep Learning

## Assignment 2

In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


## Problem 1

### Flatten data and one-hot encoding

In [4]:
image_size = 28
num_labels = 10
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


### Build computation graph and load data

In [5]:
#With gradient descent training, even this much data is prohibitive.
#Sutset the data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    #Input data
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training, validation, test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

### Run the computation and iterate

In [6]:
num_steps = 8001

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph = graph) as session:
    #Initialize the parameters: random weights and zero biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 1000 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.313282
Training accuracy: 8.8%
Validation accuracy: 11.3%
Loss at step 1000: 1.028098
Training accuracy: 79.7%
Validation accuracy: 75.6%
Loss at step 2000: 0.742318
Training accuracy: 83.3%
Validation accuracy: 76.8%
Loss at step 3000: 0.613892
Training accuracy: 85.0%
Validation accuracy: 77.2%
Loss at step 4000: 0.540180
Training accuracy: 86.2%
Validation accuracy: 77.4%
Loss at step 5000: 0.491988
Training accuracy: 87.2%
Validation accuracy: 77.5%
Loss at step 6000: 0.457632
Training accuracy: 88.0%
Validation accuracy: 77.7%
Loss at step 7000: 0.431532
Training accuracy: 88.7%
Validation accuracy: 77.8%
Loss at step 8000: 0.410829
Training accuracy: 89.2%
Validation accuracy: 77.8%
Test accuracy: 85.1%


### Stochastic Gradient Descent

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    #Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for training, validation, test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 8001

with tf.Session(graph = graph)  as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        #Pick an offset within the training data.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        #feed dictionary
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.641422
Minibatch accuracy: 10.2%
Validation accuracy: 11.8%
Minibatch loss at step 1000: 1.339463
Minibatch accuracy: 76.6%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.841011
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.680946
Minibatch accuracy: 80.5%
Validation accuracy: 78.8%
Minibatch loss at step 4000: 1.021965
Minibatch accuracy: 72.7%
Validation accuracy: 78.9%
Minibatch loss at step 5000: 0.670173
Minibatch accuracy: 80.5%
Validation accuracy: 80.1%
Minibatch loss at step 6000: 0.818149
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 7000: 0.670169
Minibatch accuracy: 78.9%
Validation accuracy: 79.2%
Minibatch loss at step 8000: 0.549293
Minibatch accuracy: 82.8%
Validation accuracy: 80.6%
Test accuracy: 87.7%


## Problem 2

### Relu and SGD

In [9]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    #Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation.
    layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    layer_2 = tf.matmul(layer_1, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = layer_2))
    
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training, validation, testing data.
    train_prediction = tf.nn.softmax(layer_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [10]:
num_steps =8001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        #Pick an offset within training data.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #Prapare feed dictionary.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 377.625702
Minibatch accuracy: 7.8%
Validation accuracy: 33.5%
Minibatch loss at step 1000: 8.159749
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 3.033492
Minibatch accuracy: 82.8%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 4.189450
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 4000: 4.508278
Minibatch accuracy: 78.1%
Validation accuracy: 81.7%
Minibatch loss at step 5000: 0.997485
Minibatch accuracy: 89.8%
Validation accuracy: 83.8%
Minibatch loss at step 6000: 2.415808
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 7000: 0.599010
Minibatch accuracy: 89.8%
Validation accuracy: 83.9%
Minibatch loss at step 8000: 1.029876
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Test accuracy: 90.1%
